# Inertia calculation

<img src="images/Robophysical Diagram.png">

In order to accurately simulate the Resonant Flapper system, we need to know the inertia of the full system. In this document, I will compute and record the total inertia.

## Some assumptions:

1. The inertia of the silicone element itself is negligible compared to the that rest of the system, so its motion during torsion can be ignored

2. The torque of the motor is large enough that we can neglect the inertia of the motor rotor in the dynamics

3. The added mass from the fluid around the wing is determined by a cylinder with a length and diameter defined by the length and width of the wing

4. The inertia of the timing belt is negligible

## Components

- Elastic element coupler
    - Made of steel and acrylic
- 1/2" x 9" steel shaft
- 1/2" x 18" steel shaft
- 1/4" x 12" steel shaft
- Aluminum belt pulley (x3)
- 1/2" to 1/4" shaft coupler
- Acrylic wing
    - 10 cm x 3.6 cm x 0.635 cm
- Added mass
    - Cylinder with
        - Diam  = 3.6 cm
        - Height = 10 cm
        


In [1]:
import math

# Elastic coupler calculations

# masses & dimensions
mass_flangecoupler = 0.1847   # kg
mass_acrylicpart  = 0.0378    # kg
mass_nutAndBolt = 0.0114      # kg
rad_flange_1 = 0.01583        # m
rad_flange_2 = 0.028315       # m
rad_acrylic  = 0.04185        # m
rad_inner = 0.00635           # m
rad_screws = 0.032            # m
massfrac1 = 0.28
massfrac2 = 0.72


The elastic coupler consists of an acrylic disk and a steel flange coupler with 6 bolts (and the associated nuts). The disk is modeled as a solid disk, and the flange coupler is modeled as 2 thick-walled cylinders

In [2]:
# Compute Inertia

# Flange calcs

I_flange1 = 1/2*(mass_flangecoupler*massfrac1)*(rad_inner**2 + rad_flange_1**2)    #kg m^2
I_flange2 = 1/2*(mass_flangecoupler*massfrac2)*(rad_inner**2 + rad_flange_2**2)    #kg m^2

I_flange = I_flange1 + I_flange2    #kg m^2

# Acrylic calcs

I_acrylic = 1/2*(mass_acrylicpart)*(rad_acrylic)**2   #kg m^2

# Bolts and nuts

I_boltsAndNuts = 6*(mass_nutAndBolt)*rad_screws**2

# Total

I_elastic_coupler = I_flange+I_acrylic+I_boltsAndNuts

print("Inertia of the elastic element coupler:", I_elastic_coupler, "kg m^2")



Inertia of the elastic element coupler: 0.0001666562404499 kg m^2


Next we'll get the moments of inertia for the shafts. We'll use a density of steel of 7700 kg/m^3

In [3]:
# Shafts

density_steel = 7700    # kg/m^3
rad_05  =  0.00635       # m
rad_025 =  0.003175     # m
len1 = 0.4572           # m
len2 = 0.2286           # m
len3 = 0.3556           # m

# volumes

vol1 = math.pi*rad_05**2*len1   #m^3
vol2 = math.pi*rad_05**2*len2   #m^3
vol3 = math.pi*rad_025**2*len3   #m^3

# masses

mass1 = vol1*density_steel
mass2 = vol2*density_steel
mass3 = vol3*density_steel

# inertias

I_shaft05_18  = 1/2*(mass1)*(rad_05)**2
I_shaft05_9   = 1/2*(mass2)*(rad_05)**2
I_shaft025_14 = 1/2*(mass3)*(rad_025)**2

#,"\n","Inertia of the 1/2 x 9",I_shaft05_9,,"\n","Inertia of the 1/4 x 14",I_shaft025_14

print(" Inertia of the 1/2 x 18:",I_shaft05_18,"kg m^2","\n","Inertia of the 1/2 x 9:",I_shaft05_9,"kg m^2","\n","Inertia of the 1/4 x 14:",I_shaft025_14,"kg m^2")


 Inertia of the 1/2 x 18: 8.991077167577818e-06 kg m^2 
 Inertia of the 1/2 x 9: 4.495538583788909e-06 kg m^2 
 Inertia of the 1/4 x 14: 4.370662512016996e-07 kg m^2


Now we can do the aluminum belt pulleys. These I will model as solid disks.

In [4]:
# Belt pulleys

mass_pulley = 0.3     # kg
rad_pulley = 0.078/2  # m

# Inertia

I_pulley = 1/2*mass_pulley*rad_pulley**2    # kg m^2

I_pulley_total = 3*I_pulley    # kg m^2

print("The total inertia of the pulleys is",I_pulley_total,"kg m^2")

The total inertia of the pulleys is 0.00068445 kg m^2


In [5]:
# 1/2" to 1/4" shaft coupler

mass_coupler = 0.24      # kg
rad_coupler = 0.015875   # m

# inertia

I_coupler = 1/2*mass_coupler*rad_coupler**2


print("The inertia of the  coupler is",I_coupler,"kg m^2")



The inertia of the  coupler is 3.0241875e-05 kg m^2


The acrylic wing is roughly a rectangular prism rotating about an axis coincident with one end. I'll use:

<img src="images/momentRectang.jpg">

With the Parallel Axis theorem to get the moment of inertia


In [6]:
# Wing

L_wing = 0.1        # m
H_wing = 0.036      # m
T_wing = 0.00635    # m
mass_wing = 0.0355  # kg


# The rotation axis is parallel to H and the COM is at L/2, T/2

I_wing_com = 1/12*mass_wing*(L_wing**2 + T_wing**2)

# parallel axis theorem

I_wing = I_wing_com + mass_wing*(L_wing/2)**2

print("The inertia of the wing is",I_wing,"kg m^2")


The inertia of the wing is 0.00011845262072916669 kg m^2


The added mass can be roughly estimated by a cylinder with height $L_{wing}$ and diameter $H_{wing}$ and mass determined by the density of the fluid.

The inertia may be calculated from:

<img src="images/momentCyl.jpg">

I will compute the moment of inertia about the $y$ axis and then use the parallel axis theorem again

In [7]:
# Added mass

height_cyl = L_wing    # m
rad_cyl = H_wing/2     # m
density_water = 997    # kg/m^3

vol_cyl = math.pi*rad_cyl**2*height_cyl   # m^3
mass_cyl = density_water*vol_cyl          # kg

# inertia

I_addedmass_COM = 1/12*mass_cyl*(3*rad_cyl**2+height_cyl**2)

I_addedmass = I_addedmass_COM + mass_cyl*(height_cyl/2)**2

print("The inertia of the added mass is",I_addedmass,"kg m^2")


The inertia of the added mass is 0.0003464941919407354 kg m^2


## Total Inertia

Now that we've calculated all of the individual inertias, we can add them together to get the total inertia

In [8]:
# Sum of inertias

I_system = I_elastic_coupler +I_shaft05_18 + I_shaft05_9 + I_shaft025_14 + I_pulley_total + I_coupler + I_wing + I_addedmass

print("The inertia of the total system is",I_system,"kg m^2")

The inertia of the total system is 0.0013602186101223704 kg m^2


## Inertia disk

Sometimes we add a large aluminum disk to the system to increase the total inertia, so it's necessary to calculate the inertia of that component. It consists of:

- A flange coupler (as in the elastic coupler above)
- A 6" diameter aluminum disk (1/4" thick) with a hole in the center with a 0.625" diameter
- 6 1/4" x 1.75" steel shafts at a radius of 2.5"



In [9]:
# inertial disk calc

density_alum = 2700     # kg/m^3
rad_outer_disk  = 0.0762     # m
rad_inner_disk = 0.0079375   # m
thickness = 0.00635     # m
len_rods = 0.04445      # m
rad_rods = 0.0635       # m

vol_disk = math.pi * (rad_outer_disk**2 - rad_inner_disk**2)*thickness
mass_disk = density_alum*vol_disk

vol_rod = math.pi*(rad_025**2)*len_rods
mass_rod = density_steel*vol_rod


# Inertia

I_disk = 1/2*mass_disk*(rad_inner_disk**2+rad_outer_disk**2) + 6*mass_rod*rad_rods**2 + I_flange

print("The inertia of the inertial disk is",I_disk,"kg m^2")




The inertia of the inertial disk is 0.0012336276290129775 kg m^2
